In [ ]:
from CC.trainer import NERTrainer

args = {
    'num_epochs': 30,
    'num_gpus': [0, 1, 2, 3],
    'bert_config_file_name': './model/chinese_wwm_ext/config.json',
    'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'hidden_dim': 300,
    'max_seq_length': 150,
    'max_scan_num': 1000000,
    'train_file': 'data/ontonotes5/train.json',
    'eval_file': 'data/ontonotes5/dev.json',
    'test_file': 'data/ontonotes5/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': 'data/ontonotes5/ontonotes5_labels.txt',
    'output_eval': True,
    'loader_name': 'le_loader',
    "word_embedding_file": "./data/tencent/word_embedding.txt",
    "word_vocab_file": "./data/tencent/tencent_vocab.txt",
    "default_tag": "O",
    'batch_size': 64,
    'eval_batch_size': 512,
    'do_shuffle': True,
    'model_name': 'LEBert',
    'task_name': 'super_predict_model'
}

from CC.predicter import NERPredict
import json

# 使用了预训练模型
args["lstm_crf_model_file"] = "/home/lpc/repos/ccNERx/save_model/ontonotes5/lstm_crf/lstm_crf_13823.pth"
args["bert_model_file"] = "/home/lpc/repos/ccNERx/save_model/ontonotes5/LEBert/LEBert_13823.pth"
predict = NERPredict(**args)


In [ ]:
import torch
text = "九十年代"
print(predict([text]))
hidden_state,masks = predict.p([text])

crf = predict.birnncrf.crf
features,masks = predict.birnncrf._BiRnnCrf__build_features(hidden_state,masks)

features = crf.fc(features)
L = features.size(1)
masks_ = masks[:, :L].float()

tag_vocab = predict.dataloader()["tag_vocab"]
labels = tag_vocab.item2idx.keys()
labels_set = set()
for label in labels:
    labels_set.add(label.split("-")[-1])

from CC.loaders.utils.label import get_labels
# 构造标签集合

ans = []
for label in labels_set:
    paths = []
    if label != "O":
        paths = ["O"] + get_labels(label,len(text),has_end=False,middle_symbol="I")
    paths += ["O"] * (150-len(paths))
    # print(paths)
    paths = torch.tensor(tag_vocab.token2id(paths),dtype=torch.long).unsqueeze(0)
    ans.append(crf._CRF__score_sentence(features,paths[:, :L].long(),masks_).item())
    # print(crf.loss(features,paths,masks))
soft = torch.nn.Softmax(-1)
ans = soft(torch.tensor(ans))
for label,ans in zip(labels_set,ans):
    print(label,round(ans.item(),6))